In [ ]:
import math
import torch
import roma

In [75]:
def euler_xyz_to_zxy(euler):
    return torch.tensor([euler[2], euler[0], euler[1]])

def euler_zxy_to_xyz(euler):
    return torch.tensor([euler[1], euler[2], euler[0]])

print(roma.euler_to_unitquat('zxy', euler_xyz_to_zxy(torch.tensor([15, 22, -45])), degrees=True))
print(euler_zxy_to_xyz(roma.unitquat_to_euler('zxy', torch.tensor([0.04598, 0.22381, -0.39545, 0.88962]), degrees=True)))

tensor([ 0.0460,  0.2238, -0.3954,  0.8896])
tensor([ 14.9999,  22.0000, -45.0000])


In [78]:
def angle_normalize(angle):
    return (angle + 180) % 360 - 180

def euler_normalize(euler):
    # Normalize each axis to be between -180 and 180
    return torch.tensor([angle_normalize(euler[0]), angle_normalize(euler[1]), angle_normalize(euler[2])])

def get_camera_position_and_rotation(character_pos, character_rot_y, camera_pos, camera_rot, distance):
    convention = 'zxy'
    character_rot = torch.tensor([0.0, character_rot_y, 0.0])
    camera_rot[1] = -camera_rot[1]
    
    character_rotation = roma.euler_to_unitquat(convention, euler_xyz_to_zxy(character_rot), degrees=True)
    print("character_rotation", character_rotation)
    local_camera_rotation = roma.euler_to_unitquat(convention, euler_xyz_to_zxy(camera_rot), degrees=True)
    print("local_camera_rotation", local_camera_rotation)
    extra_rotation = roma.euler_to_unitquat(convention, euler_xyz_to_zxy(torch.tensor([0.0, 180, 0.0])), degrees=True)
    
    combined_rotation = roma.quat_product(roma.quat_product(character_rotation, local_camera_rotation), extra_rotation)
    print("combined_rotation", combined_rotation)
    
    world_camera_pos = character_pos + roma.quat_action(character_rotation, camera_pos)
    print("world_camera_pos", world_camera_pos)
    camera_backward = roma.quat_action(combined_rotation, torch.tensor([0.0, 0.0, -distance]))
    print("camera_backward", camera_backward)
    final_camera_position = world_camera_pos + camera_backward
    print("final_camera_position", final_camera_position)
    
    final_camera_rotation = euler_zxy_to_xyz(roma.unitquat_to_euler(convention, combined_rotation, degrees=True))
    print("final_camera_rotation", final_camera_rotation)
    
    return final_camera_position, final_camera_rotation

character_pos = torch.tensor([0.5, 0, -0.5])
character_rot_y = 160
camera_pos = torch.tensor([1, 2, -0.8])
camera_rot = torch.tensor([10, 20, 45])
camera_distance = 2

get_camera_position_and_rotation(character_pos, character_rot_y, camera_pos, camera_rot, camera_distance)

character_rotation tensor([0.0000, 0.9848, 0.0000, 0.1736])
local_camera_rotation tensor([ 0.0131, -0.1927,  0.3894,  0.9006])
combined_rotation tensor([-0.0547,  0.3461,  0.3858, -0.8535])
world_camera_pos tensor([-0.7133,  2.0000, -0.0903])
camera_backward tensor([ 1.2660, -0.3473, -1.5088])
final_camera_position tensor([ 0.5527,  1.6527, -1.5991])
final_camera_rotation tensor([-10.0000, -40.0000, -45.0000])


(tensor([ 0.5527,  1.6527, -1.5991]), tensor([-10.0000, -40.0000, -45.0000]))

In [77]:
# Load test cases
import json
with open("test_cases.json", "r") as f:
    test_cases = json.load(f)

def vector3_to_tensor(vector):
    return torch.tensor([vector['x'], vector['y'], vector['z']], dtype=torch.float32)

# Run test cases
for i, test_case in enumerate(test_cases):
    character_pos = vector3_to_tensor(test_case["characterPos"])
    character_rot_y = test_case["characterRotY"]
    camera_pos = vector3_to_tensor(test_case["cameraPos"])
    camera_rot = vector3_to_tensor(test_case["cameraRot"])
    distance = test_case["cameraDistance"]
    expected_pos = vector3_to_tensor(test_case["pos"])
    expected_rot = vector3_to_tensor(test_case["rot"])
    
    pos, rot = get_camera_position_and_rotation(character_pos, character_rot_y, camera_pos, camera_rot, distance)
    assert torch.allclose(pos, expected_pos, atol=1e-3), f"Test case {i + 1} failed, pos expected {expected_pos}, got {pos}"
    assert torch.allclose(euler_normalize(rot), euler_normalize(expected_rot), atol=1e-3), f"Test case {i + 1} failed, rot expected {expected_rot}, got {rot}"
    print(f"Test case {i+1} passed")

character_rotation tensor([0.0000, 0.2665, 0.0000, 0.9638])
local_camera_rotation tensor([ 0.2988, -0.5251,  0.7451,  0.2827])
combined_rotation tensor([-0.6385,  0.4124,  0.4865,  0.4307])
world_camera_pos tensor([-3.4904, -9.1749, -1.7670])
camera_backward tensor([ 1.8102, -6.4727,  1.0583])
final_camera_position tensor([ -1.6802, -15.6476,  -0.7087])
final_camera_rotation tensor([ -72.0501, -120.3119, -159.5769])
Test case 1 passed
character_rotation tensor([0.0000, 0.3016, 0.0000, 0.9534])
local_camera_rotation tensor([ 0.2662, -0.4347,  0.8243,  0.2464])
combined_rotation tensor([-0.7056,  0.3660,  0.5024,  0.3402])
world_camera_pos tensor([-3.6586, -9.4851, -1.9613])
camera_backward tensor([ 3.3042, -6.0900,  1.8949])
final_camera_position tensor([ -0.3545, -15.5751,  -0.0664])
final_camera_rotation tensor([ -57.9762, -119.8336, -160.7551])
Test case 2 passed
character_rotation tensor([0.0000, 0.3362, 0.0000, 0.9418])
local_camera_rotation tensor([ 0.2274, -0.3358,  0.8893,  0.21